In [ ]:
"""
Primer intento: MONOMODO
--> MFCC
--> KENLM (5 grams)
--> parallel args = None
--> mfccConfig with energy

"""

In [1]:
path_dataset ='/home/marrakchi/TFM/TFM_env/Datasets/ExKaldiASRDatasetFormat/' 

In [2]:
# Systems Libraries
import os
import time

# For audio processing
import librosa
import librosa.display
import IPython as ipd
from IPython.display import display, HTML # For displaying tables

#from torchsummary import summary

# For data processing
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#import sox

# For Neural networks
import shutil,socket
import tensorflow as tf
from tensorflow import keras
import random
import torch
from torch import nn
import pickle as pkl
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# For text processing
import string
from datetime import datetime

# Import exkaldi package
import exkaldi
from exkaldi import declare
exkaldi.info.reset_kaldi_root('/kaldi')

exkaldi.info.reset_kaldi_root( yourPath )
If not, ERROR will occur when implementing some core functions.


In [3]:
# ExKaldi Configuration
ExKaldiInfo = exkaldi.info
print(ExKaldiInfo)
print(ExKaldiInfo.KALDI)
print(ExKaldiInfo.KALDI_ROOT)

ExKaldiInfo(version='1.3.5.4', major='1', minor='3', patch='5', upload='4')
Kaldi(version='5.5', major='5', minor='5')
/kaldi


In [4]:
# Ensuring that GPU are detected/available and ready to use
print(torch.cuda.device_count())
print(torch.cuda.is_available())

1
True


In [5]:
print(tf.__version__)

2.5.0


# Load Data

In [83]:
# Read txt file as a list
mi_path = path_dataset+"wav_names.txt"
file = open(mi_path, 'r') 
text = file.read() 
wav_list = text.split("\n")
file.close()

print(len(wav_list))

123956


In [26]:
wav_list.sort()
print(wav_list[:5])

['0000df16-47ea-428f-8367-df2ce365d5c4.wav', '000127f7-2705-4135-b16c-04daaa80e88d.wav', '0001d4a5-e7f1-42a4-b71f-74bd1549f2d4.wav', '000275c9-1aa3-4112-81df-0883a86d3fb7.wav', '0002a149-56cf-4ce0-b2c8-506a53b47012.wav']


In [5]:
# Load lexicon
lexicons = exkaldi.load_lex(os.path.join(path_dataset, "dict", "lexicons.lex"))
print(lexicons)
# Load transcriptions
trans = exkaldi.load_transcription(os.path.join(path_dataset,"text"))
trainTrans = exkaldi.load_transcription(os.path.join(path_dataset,'data/train',"text"))
valTrans = exkaldi.load_transcription(os.path.join(path_dataset,'data/val',"text"))
testTrans = exkaldi.load_transcription(os.path.join(path_dataset,'data/test',"text"))
# Load wavScp
wavScp = exkaldi.load_list_table(os.path.join(path_dataset,"wav.scp") )
wavScp_train = exkaldi.load_list_table(os.path.join(path_dataset,'data/train',"wav.scp") )
wavScp_val = exkaldi.load_list_table(os.path.join(path_dataset,'data/val',"wav.scp") )
wavScp_test = exkaldi.load_list_table(os.path.join(path_dataset,'data/test',"wav.scp") )

# Compute MFCC

In [6]:
# Parameters
parallel = 1
mfccConfig={"--use-energy":"true"}
#mfccConfig={"--use-energy":"false"}

In [49]:
print(ExKaldiInfo.timeout)
ExKaldiInfo.set_timeout(28800) # 1/2 hour
print(ExKaldiInfo.timeout)

1800
28800


In [8]:
# Training without parallel processes
#------------------- Train -------------------------------------
"""wavFiles = exkaldi.utils.split_txt_file(
                                os.path.join(path_dataset,"data","train","wav.scp"), 
                                chunks=parallel,
                            )
"""
feat = exkaldi.compute_mfcc(
                        os.path.join(path_dataset,"data","train","wav.scp"),
                        config=mfccConfig
                        #outFile=os.path.join(path_dataset,"mfcc","train","raw_mfcc.ark")
                    )
#feat = exkaldi.merge_archives(feats) # Merge multiple archives to one
#feat = feat.fetch(arkType='feat')
feat.save( os.path.join(path_dataset,"mfcc","train","raw_mfcc.ark") )
print(f"Generate TRAIN raw MFCC feature done.")

Generate TRAIN raw MFCC feature done.


In [89]:
# 1. compute feature (MFCC)
#------------------- Train -------------------------------------
wavFiles = exkaldi.utils.split_txt_file(
                                os.path.join(path_dataset,"data","train","wav.scp"), 
                                chunks=parallel,
                            )
feats = exkaldi.compute_mfcc(
                        wavFiles, 
                        config=mfccConfig,
                        outFile=os.path.join(path_dataset,"mfcc","train","raw_mfcc.ark")
                    )
feat = exkaldi.merge_archives(feats) # Merge multiple archives to one
feat = feat.fetch(arkType='feat')
feat.save( os.path.join(path_dataset,"mfcc","train","raw_mfcc.ark") )
print(f"Generate TRAIN raw MFCC feature done.")

#-------------------- Val --------------------------------------
feat2 = exkaldi.compute_mfcc(
                        os.path.join(path_dataset,"data",'val',"wav.scp"), 
                        config=mfccConfig,
                    )
feat2.save( os.path.join(path_dataset,"mfcc","val","raw_mfcc.ark") )
print(f"Generate VAL raw MFCC feature done.")

#-------------------- Test -------------------------------------
feat3 = exkaldi.compute_mfcc(
                        os.path.join(path_dataset,"data","test","wav.scp"), 
                        config=mfccConfig,
                    )
feat3.save( os.path.join(path_dataset,"mfcc","test","raw_mfcc.ark") )

print(f"Generate TEST raw MFCC feature done.")

Generate TRAIN raw MFCC feature done.
Generate VAL raw MFCC feature done.
Generate TEST raw MFCC feature done.


In [6]:
for Name in ["train", "val", "test"]:
    feat = exkaldi.load_index_table(os.path.join(path_dataset,"mfcc",Name,"raw_mfcc.ark"))
    print(Name+' feat loaded')
    # Compute CMVN
    print('Start CMVN')
    cmvn = exkaldi.compute_cmvn_stats(
                                    feat=feat,
                                    spk2utt=os.path.join(path_dataset,"data",Name,"spk2utt"),
                                )
    cmvn.save( os.path.join(path_dataset,"mfcc",Name,"cmvn.ark") )
    print(f"Generate CMVN statistics for "+ Name +" done.")
    # Apply CMVN
    feat = exkaldi.use_cmvn(
                        feat=feat,
                        cmvn=cmvn,
                        utt2spk=os.path.join(path_dataset,"data",Name,"utt2spk"),
                    )
    feat.save(os.path.join(path_dataset,"mfcc",Name,"mfcc_cmvn.ark"))
    print(f"Generate MFCC feature (applied CMVN) for "+ Name +" done.")


train feat loaded
Start CMVN
Generate CMVN statistics for train done.
Generate MFCC feature (applied CMVN) for train done.
val feat loaded
Start CMVN
Generate CMVN statistics for val done.
Generate MFCC feature (applied CMVN) for val done.
test feat loaded
Start CMVN
Generate CMVN statistics for test done.
Generate MFCC feature (applied CMVN) for test done.


# Useful functions

In [7]:
def make_WFST_graph(outDir, hmm, tree):

    print("Start to make WFST graph.")
    exkaldi.utils.make_dependent_dirs(outDir, pathIsFile=False)
    print(f"Load lexicon bank.")
    lexicons = exkaldi.load_lex(os.path.join(path_dataset,"dict","lexicons.lex"))
    
    # iLabel file will be generated in this step.
    _, ilabelFile = exkaldi.decode.graph.compose_CLG(
                                            lexicons,
                                            tree,
                                            os.path.join(path_dataset,"lm",f"LG.{order}.fst"),
                                            outFile=os.path.join(outDir,f"CLG.{order}.fst"),
                                        )
    print(f"Generate CLG fst done.")
    exkaldi.decode.graph.compose_HCLG(
                                    hmm,
                                    tree,
                                    CLGFile=os.path.join(outDir,f"CLG.{order}.fst"),
                                    iLabelFile=ilabelFile,
                                    outFile=os.path.join(outDir,f"HCLG.{order}.fst"),
                                )
    print(f"Compose HCLG fst done.")

def GMM_decode_mfcc_and_score(outDir, hmm, HCLGfile, tansformMatFile=None):

    exkaldi.utils.make_dependent_dirs(outDir, pathIsFile=False)

    lexicons = exkaldi.load_lex(os.path.join(path_dataset,"dict","lexicons.lex"))
    print(f"Load test feature.")
    featFile = os.path.join(path_dataset,"mfcc","test","mfcc_cmvn.ark")
    feat = exkaldi.load_feat(featFile)
    if tansformMatFile is None:
        print(f"Feature type is delta. Add {delta}-order deltas.")
        feat = feat.add_delta(order=delta)
    else:
        print(f"Feature type is lda+mllt. Transform feature.")
        feat = feat.splice(left=splice,right=splice)
        feat = exkaldi.transform_feat(feat, tansformMatFile)
    
    if parallel > 1:
        feat = feat.subset(chunks=parallel)

    print("Start to decode")
    st = time.time()
    lat = exkaldi.decode.wfst.gmm_decode(
                                    feat, 
                                    hmm, 
                                    HCLGfile, 
                                    symbolTable=lexicons("words"),
                                    beam=beam, 
                                    latBeam=latBeam, 
                                    acwt=acwt,
                                    outFile=os.path.join(outDir,"test.lat"),
                                )
    print("Decode time cost: ","%.2f"%(time.time()-st),"seconds")
    if isinstance(lat,list):
        lat = exkaldi.merge_archives(lat)

    print(f"Generate lattice done.")
    
    refText = exkaldi.load_transcription(os.path.join(path_dataset,"data","test","text"))
    # Transform transcriptions from words to phones
    word2phone = exkaldi.load_list_table(path_dataset+'word2phone.map')
    refText = refText.convert(word2phone, unkSymbol='<UNK>')

    print("Now score:")
    bestWER = (1000, 0, 0)
    bestResult = None
    for penalty in [0., 0.5, 1.0]:
        for LMWT in range(1, 11):
            # Add penalty
            newLat = lat.add_penalty(penalty)
            # Get 1-best result (phone-level)
            result = newLat.get_1best(lexicons("phones"), hmm, lmwt=LMWT, acwt=1, phoneLevel=True)
            # Transform from int value format to text format
            result = exkaldi.hmm.transcription_from_int(result, lexicons("phones"))
            # Compute PER
            score = exkaldi.decode.score.wer(ref=refText, hyp=result, mode="present")
            if score.WER < bestWER[0]:
                bestResult = result
                bestWER = (score.WER, penalty, LMWT)
            print(f"Penalty: {penalty}, LMWT: {LMWT}, PER: {score.WER}%")

    print("Score done. Saved the best result.")
    bestResult.save(os.path.join(outDir, "hyp.txt") )
    with open(os.path.join(outDir,"best_PER"),"w") as fw:
        fw.write( f"PER {bestWER[0]}, penalty {bestWER[1]}, LMWT {bestWER[2]}" )

In [8]:
def GMM_decode_fmllr_and_score(outDir, hmm, HCLGfile, tansformMatFile=None):
    exkaldi.utils.make_dependent_dirs(outDir, pathIsFile=False)
    lexicons = exkaldi.load_lex(os.path.join(path_dataset,"dict","lexicons.lex"))
    print(f"Load test feature.")
    featFile = os.path.join(path_dataset,"mfcc","test","mfcc_cmvn.ark")
    feat = exkaldi.load_feat(featFile)
    if tansformMatFile is None:
        print("Feature type is delta. Add 2-order deltas.")
        feat = feat.add_delta(order=delta)
        feat = feat.save(os.path.join(outDir,"test_mfcc_cmvn_delta.ark"),returnIndexTable=True)
    else:
        print("Feature type is lda+mllt")
        feat = feat.splice(left=splice,right=splice)
        print("Transform feature")
        feat = exkaldi.transform_feat(feat, tansformMatFile)
        feat = feat.save(os.path.join(outDir,"test_mfcc_cmvn_lda.ark"),returnIndexTable=True)

    ## 1. Estimate the primary transform matrix from alignment or lattice.
    ## We estimate it from lattice, so we decode it firstly.
    print("Decode the first time with original feature.")
    preLat = exkaldi.decode.wfst.gmm_decode(
                                        feat, 
                                        hmm, 
                                        HCLGfile, 
                                        symbolTable=lexicons("words"),
                                        beam=10, 
                                        latBeam=6, 
                                        acwt=acwt,
                                        maxActive=2000,
                                    )
    preLat.save(os.path.join(outDir,"test_premary.lat"))

    print("Estimate the primary fMLLR transform matrix.")
    preTransMatrix = exkaldi.hmm.estimate_fMLLR_matrix(
                                        aliOrLat=preLat,
                                        lexicons=lexicons,
                                        aliHmm=hmm, 
                                        feat=feat,
                                        adaHmm=None,
                                        silenceWeight=0.01,
                                        acwt=acwt,
                                        spk2utt=os.path.join(path_dataset,"data","test","spk2utt"),
                                    )
    del preLat
    ## 2. Transform feature. We will use new feature to estimate the secondary transform matrix from lattice.
    print("Transform feature with primary matrix.")
    fmllrFeat = exkaldi.use_fmllr(
                        feat,
                        preTransMatrix,
                        utt2spk=os.path.join(path_dataset,"data","test","utt2spk"),
                    )
    print("Decode the second time with primary fmllr feature.")
    secLat = exkaldi.decode.wfst.gmm_decode(
                                        fmllrFeat, 
                                        hmm, 
                                        HCLGfile, 
                                        symbolTable=lexicons("words"),
                                        beam=beam,
                                        latBeam=latBeam,
                                        acwt=acwt,
                                        maxActive=7000,
                                        config={"--determinize-lattice":"false"},
                                    )
    print("Determinize secondary lattice.")
    thiLat = secLat.determinize(acwt=acwt, beam=4)
    print("Estimate the secondary fMLLR transform matrix.")
    secTransMatrix = exkaldi.hmm.estimate_fMLLR_matrix(
                                        aliOrLat=thiLat,
                                        lexicons=lexicons,
                                        aliHmm=hmm, 
                                        feat=fmllrFeat,
                                        adaHmm=None,
                                        silenceWeight=0.01,
                                        acwt=acwt,
                                        spk2utt=os.path.join(path_dataset,"data","test","spk2utt"),
                                    )
    del fmllrFeat
    del thiLat
    ## 3. Compose the primary matrix and secondary matrix and get the final transform matrix.
    print("Compose the primary and secondary transform matrix.")
    finalTransMatrix = exkaldi.hmm.compose_transform_matrixs(
                                                        matA=preTransMatrix,
                                                        matB=secTransMatrix,
                                                        bIsAffine=True,
                                                    )
    finalTransMatrix.save(os.path.join(outDir,"trans.ark"))
    print("Transform feature with final matrix.")
    ## 4. Transform feature with the final transform matrix and use it to decode.
    ## We directly use the lattice generated in the second step. The final lattice is obtained.
    finalFmllrFeat = exkaldi.use_fmllr(
                                    feat,
                                    finalTransMatrix,
                                    utt2spk=os.path.join(path_dataset,"data","test","utt2spk"),
                                )
    del finalTransMatrix
    print("Rescore secondary lattice.")
    lat = secLat.am_rescore(
                        hmm=hmm,
                        feat=finalFmllrFeat,
                    )
    print("Determinize secondary lattice.")
    lat = lat.determinize(acwt=acwt, beam=6)
    lat.save(os.path.join(outDir,"test.lat"))
    print("Generate lattice done.")

    #phoneMapFile = os.path.join(path_dataset,"dict","phones.48_to_39.map")
    #phoneMap = exkaldi.load_list_table(phoneMapFile,name="48-39")
    #refText = exkaldi.load_transcription(os.path.join(args.expDir,"data","test","text")).convert(phoneMap)
    
    refText = exkaldi.load_transcription(os.path.join(path_dataset,"data","test","text"))
    # Transform transcriptions from words to phones
    word2phone = exkaldi.load_list_table(path_dataset+'word2phone.map')
    refText = refText.convert(word2phone, unkSymbol='<UNK>')
    
    refText.save(os.path.join(outDir,"ref.txt") )
    print("Generate reference text done.")

    print("Now score:")
    bestWER = (1000, 0, 0)
    bestResult = None
    for penalty in [0., 0.5, 1.0]:
        for LMWT in range(1, 11):
            # Add penalty
            newLat = lat.add_penalty(penalty)
            # Get 1-best result (phone-level)
            result = newLat.get_1best(lexicons("phones"), hmm, lmwt=LMWT, acwt=1, phoneLevel=True)
            # Transform from int value format to text format
            result = exkaldi.hmm.transcription_from_int(result, lexicons("phones"))
            # Transform 48-phones to 39-phones
            #result = result.convert(phoneMap, None)
            # Compute PER
            score = exkaldi.decode.score.wer(ref=refText, hyp=result, mode="present")
            if score.WER < bestWER[0]:
                bestResult = result
                bestWER = (score.WER, penalty, LMWT)
            print(f"Penalty: {penalty}, LMWT: {LMWT}, PER: {score.WER}%")
    print("Score done. Save the best result.")
    bestResult.save(os.path.join(outDir, "hyp.txt") )
    with open(os.path.join(outDir,"best_PER"),"w") as fw:
        fw.write( f"PER {bestWER[0]}, penalty {bestWER[1]}, LMWT {bestWER[2]}" )

# Train monophone GMM-HMM model

In [5]:
# ARGS
delta = 2 #(n-order to add to the feature)
numIters = 40 # Training iterations
maxIterInc = 30 # Final iteration of increasing gaussians
realignIter = [1,2,3,4,5,6,7,8,9,10,12,14,16,18,20,23,26,29,32,35,38] # iteration to allign feature
order = 5 # N-GRAM to use (min 1 | max 6)
beam = 13 # Decode be                                          am size
latBeam = 6 # Lattice beam size
acwt = 0.083333 # Acoustic model weight
parallel = 1 # parallel process to compute feature of train dataset (min 1 | max 10)
skipTrain = False # If True, skip training. Do decoding only
splice = 3 # How many left-right frames to splice
mlltIter = [2,4,6,12] # The iteration estimate mllt matrix

In [8]:
# ------------- Prepare feature for training ----------------------
# 1. Load the feature for training (We use the index table format)
feat = exkaldi.load_index_table(os.path.join(path_dataset,"mfcc","train","mfcc_cmvn.ark"))
print(f"Load MFCC+CMVN feature.")
feat = exkaldi.add_delta(feat, order=delta, outFile=os.path.join(path_dataset,"train_mono","mfcc_cmvn_delta.ark"))
print(f"Add {delta}-order deltas.")
# 2. Load lexicon bank
lexicons = exkaldi.load_lex(os.path.join(path_dataset,"dict","lexicons.lex"))
print(f"Restorage lexicon bank.")

Load MFCC+CMVN feature.
Add 2-order deltas.
Restorage lexicon bank.


In [9]:
# ------------- Start training ----------------------
# 1. Initialize a monophone HMM object
model = exkaldi.hmm.MonophoneHMM(lexicons=lexicons, name="mono")
model.initialize(
            feat=feat,
            topoFile=os.path.join(path_dataset,"dict","topo")
        )
print(f"Initialized a monophone HMM-GMM model: {model.info}.")

# 2. Split data for parallel training
transcription = exkaldi.load_transcription(os.path.join(path_dataset,"data","train","text"))
transcription = transcription.sort()
if parallel > 1:
    # split feature
    feat = feat.sort(by="utt").subset(chunks=parallel)
    # split transcription depending on utterance IDs of each feature
    temp = []
    for f in feat:
        temp.append( transcription.subset(keys=f.utts) )
    transcription = temp
    
print("\n---------------------- Training -------------------------------------------------")
# 3. Train
model.train(
            feat,
            transcription, 
            LFile=os.path.join(path_dataset,"dict","L.fst"),
            tempDir=os.path.join(path_dataset,"train_mono"),
            numIters=numIters, 
            maxIterInc=maxIterInc, 
            totgauss=1000, 
            realignIter=realignIter,
            boostSilence=1.0,
        )
print("---------------------------------------------------------------------------------\n")
print(model.info)
# Save the tree
model.tree.save(os.path.join(path_dataset,"train_mono","tree"))
print(f"Tree has been saved.")

# 4. Realign with boostSilence 1.25
print("Realign the training feature (boost silence = 1.25)")
trainGraphFiles = exkaldi.utils.list_files(os.path.join(path_dataset,"train_mono","*train_graph"))
model.align(
            feat,
            trainGraphFile=trainGraphFiles,  # train graphs have been generated in the train step.
            boostSilence=1.25, #1.5
            outFile=os.path.join(path_dataset,"train_mono","final.ali")
        )
del feat
print("Save the new alignment done.")
tree = model.tree

Initialized a monophone HMM-GMM model: GmmHmmInfo(phones=26, pdfs=82, transitionIds=180, transitionStates=82, dimension=39, gaussians=82).

---------------------- Training -------------------------------------------------
Start to train monophone model.
Start Time: 2021/05/21-23:49:42
Convert transcription to int value format.
Compiling training graph.
Iter >> 0
Aligning data equally
Accumulate GMM statistics
Update GMM parameters
Used time: 377.6980 seconds
Iter >> 1
Aligning data
Accumulate GMM statistics
Update GMM parameters
Used time: 3928.3835 seconds
Iter >> 2
Aligning data
Accumulate GMM statistics
Update GMM parameters
Used time: 970.2897 seconds
Iter >> 3
Aligning data
Accumulate GMM statistics
Update GMM parameters
Used time: 903.3061 seconds
Iter >> 4
Aligning data
Accumulate GMM statistics
Update GMM parameters
Used time: 884.5455 seconds
Iter >> 5
Aligning data
Accumulate GMM statistics
Update GMM parameters
Used time: 888.8432 seconds
Iter >> 6
Aligning data
Accumulate G

In [11]:
# ------------- Compile WFST training ----------------------
# Make a WFST decoding graph
make_WFST_graph(
            outDir=os.path.join(path_dataset,"train_mono","graph"),
            hmm=model,
            tree=tree,
        )

# Decode test data
GMM_decode_mfcc_and_score(
            outDir=os.path.join(path_dataset,"train_mono",f"decode_{order}grams"), 
            hmm=model,
            HCLGfile=os.path.join(path_dataset,"train_mono","graph",f"HCLG.{order}.fst"),
        )

Start to make WFST graph.
Load lexicon bank.
Generate CLG fst done.
Compose HCLG fst done.
Load test feature.
Feature type is delta. Add 2-order deltas.
Start to decode
Decode time cost:  4503.58 seconds
Generate lattice done.
Now score:
Penalty: 0.0, LMWT: 1, PER: 30.31%
Penalty: 0.0, LMWT: 2, PER: 29.96%
Penalty: 0.0, LMWT: 3, PER: 29.64%
Penalty: 0.0, LMWT: 4, PER: 29.46%
Penalty: 0.0, LMWT: 5, PER: 29.31%
Penalty: 0.0, LMWT: 6, PER: 29.32%
Penalty: 0.0, LMWT: 7, PER: 29.49%
Penalty: 0.0, LMWT: 8, PER: 29.77%
Penalty: 0.0, LMWT: 9, PER: 30.19%
Penalty: 0.0, LMWT: 10, PER: 30.64%
Penalty: 0.5, LMWT: 1, PER: 30.33%
Penalty: 0.5, LMWT: 2, PER: 29.95%
Penalty: 0.5, LMWT: 3, PER: 29.68%
Penalty: 0.5, LMWT: 4, PER: 29.6%
Penalty: 0.5, LMWT: 5, PER: 29.48%
Penalty: 0.5, LMWT: 6, PER: 29.51%
Penalty: 0.5, LMWT: 7, PER: 29.74%
Penalty: 0.5, LMWT: 8, PER: 30.1%
Penalty: 0.5, LMWT: 9, PER: 30.6%
Penalty: 0.5, LMWT: 10, PER: 31.15%
Penalty: 1.0, LMWT: 1, PER: 30.32%
Penalty: 1.0, LMWT: 2, PER: 

In [65]:
# Test score Without CMVN (only raw mfcc)
"""outDir = os.path.join(path_dataset,"train_mono",f"decode_{order}grams")
HCLGfile=os.path.join(path_dataset,"train_mono","graph",f"HCLG.{order}.fst")

# Load lattice file
lat = exkaldi.load_lat(os.path.join(path_dataset,"train_mono",f"decode_{order}grams","test.lat"))
hmm = exkaldi.load_hmm(os.path.join(path_dataset,"train_mono","final.mdl"))
refText = exkaldi.load_transcription(os.path.join(path_dataset,"data","test","text"))

# Transform transcriptions from words to phones
word2phone = exkaldi.load_list_table(path_dataset+'word2phone.map')
refText = refText.convert(word2phone, unkSymbol='<UNK>')

print("Now score:")
bestWER = (1000, 0, 0)
bestResult = None
for penalty in [0., 0.5, 1.0]:
    for LMWT in range(1, 11):
        # Add penalty
        newLat = lat.add_penalty(penalty)
        # Get 1-best result (phone-level)
        result = newLat.get_1best(lexicons("phones"), hmm, lmwt=LMWT, acwt=1, phoneLevel=True)
        # Transform from int value format to text format
        result = exkaldi.hmm.transcription_from_int(result, lexicons("phones"))
        # Compute PER
        score = exkaldi.decode.score.wer(ref=refText, hyp=result, mode="present")
        if score.WER < bestWER[0]:
            bestResult = result
            bestWER = (score.WER, penalty, LMWT)
        print(f"Penalty: {penalty}, LMWT: {LMWT}, PER: {score.WER}%")

print("Score done. Saved the best result.")
bestResult.save(os.path.join(outDir, "hyp.txt") )
with open(os.path.join(outDir,"best_PER"),"w") as fw:
    fw.write( f"PER {bestWER[0]}, penalty {bestWER[1]}, LMWT {bestWER[2]}" )
"""

Now score:
Penalty: 0.0, LMWT: 1, PER: 30.3%
Penalty: 0.0, LMWT: 2, PER: 29.94%
Penalty: 0.0, LMWT: 3, PER: 29.61%
Penalty: 0.0, LMWT: 4, PER: 29.41%
Penalty: 0.0, LMWT: 5, PER: 29.31%
Penalty: 0.0, LMWT: 6, PER: 29.36%
Penalty: 0.0, LMWT: 7, PER: 29.46%
Penalty: 0.0, LMWT: 8, PER: 29.72%
Penalty: 0.0, LMWT: 9, PER: 30.11%
Penalty: 0.0, LMWT: 10, PER: 30.62%
Penalty: 0.5, LMWT: 1, PER: 30.32%
Penalty: 0.5, LMWT: 2, PER: 29.98%
Penalty: 0.5, LMWT: 3, PER: 29.67%
Penalty: 0.5, LMWT: 4, PER: 29.48%
Penalty: 0.5, LMWT: 5, PER: 29.46%
Penalty: 0.5, LMWT: 6, PER: 29.53%
Penalty: 0.5, LMWT: 7, PER: 29.65%
Penalty: 0.5, LMWT: 8, PER: 30.08%
Penalty: 0.5, LMWT: 9, PER: 30.53%
Penalty: 0.5, LMWT: 10, PER: 31.1%
Penalty: 1.0, LMWT: 1, PER: 30.33%
Penalty: 1.0, LMWT: 2, PER: 30.0%
Penalty: 1.0, LMWT: 3, PER: 29.76%
Penalty: 1.0, LMWT: 4, PER: 29.57%
Penalty: 1.0, LMWT: 5, PER: 29.59%
Penalty: 1.0, LMWT: 6, PER: 29.74%
Penalty: 1.0, LMWT: 7, PER: 29.97%
Penalty: 1.0, LMWT: 8, PER: 30.52%
Penalty: 1

# Train triphone GMM-HMM model

In [7]:
# ARGS
delta = 2 #(n-order to add to the feature)
numIters = 35 # Training iterations
maxIterInc = 25 # Final iteration of increasing gaussians
realignIter = [10,20,30] # iteration to allign feature
order = 5 # N-GRAM to use (min 1 | max 6)
beam = 13 # Decode be                                          am size
latBeam = 6 # Lattice beam size
acwt = 0.083333 # Acoustic model weight
parallel = 1 # parallel process to compute feature of train dataset (min 1 | max 10)
skipTrain = False # If True, skip training. Do decoding only

In [8]:
# ------------- Prepare feature and previous alignment for training ----------------------
# 1. Load the feature for training
feat = exkaldi.load_index_table(os.path.join(path_dataset,"mfcc","train","mfcc_cmvn.ark"))
print(f"Load MFCC+CMVN feature.")
feat = exkaldi.add_delta(feat, order=delta, outFile=os.path.join(path_dataset,"train_delta","mfcc_cmvn_delta.ark"))
print(f"Add {delta}-order deltas.")
# 2. Load lexicon bank
lexicons = exkaldi.load_lex(os.path.join(path_dataset,"dict","lexicons.lex"))
print(f"Restorage lexicon bank.")
# 3. Load previous alignment
ali = exkaldi.load_index_table(os.path.join(path_dataset,"train_mono","*final.ali"),useSuffix="ark")

Load MFCC+CMVN feature.
Add 2-order deltas.
Restorage lexicon bank.


In [9]:
# -------------- Build the decision tree ------------------------
print("Start build a tree")
tree = exkaldi.hmm.DecisionTree(lexicons=lexicons, contextWidth=3, centralPosition=1)
tree.train(
            feat=feat, 
            hmm=os.path.join(path_dataset,"train_mono","final.mdl"), 
            ali=ali, 
            topoFile=os.path.join(path_dataset,"dict","topo"), 
            numLeaves=2500,
            tempDir=os.path.join(path_dataset,"train_delta"), 
        )
print(f"Build tree done.")

Start build a tree
Start to build decision tree.
Start Time: 2021/05/23-00:06:37
Accumulate tree statistics
Cluster phones and compile questions
Build tree
Done to build the decision tree.
Saved Final Tree: /home/marrakchi/TFM/TFM_env/Datasets/ExKaldiASRDatasetFormat/train_delta/tree
End Time: 2021/05/23-00:09:35
Build tree done.


In [10]:
# ------------- Start training ----------------------
# 1. Initialize a triphone HMM object
model = exkaldi.hmm.TriphoneHMM(lexicons=lexicons, name="tri")
model.initialize(
            tree=tree, 
            topoFile=os.path.join(path_dataset,"dict","topo"),
            treeStatsFile=os.path.join(path_dataset,"train_delta","treeStats.acc"),
        )
print(f"Initialized a triphone HMM-GMM model: {model.info}.")

# 2. convert the previous alignment
print(f"Transform the alignment")
newAli = exkaldi.hmm.convert_alignment(
                                ali=ali,
                                originHmm=os.path.join(path_dataset,"train_mono","final.mdl"), 
                                targetHmm=model, 
                                tree=tree,
                                outFile=os.path.join(path_dataset,"train_delta","initial.ali"),
                            )

Initialized a triphone HMM-GMM model: GmmHmmInfo(phones=26, pdfs=2088, transitionIds=4258, transitionStates=2121, dimension=39, gaussians=2088).
Transform the alignment


In [11]:
# 2. Split data for parallel training
transcription = exkaldi.load_transcription(os.path.join(path_dataset,"data","train","text"))
transcription = transcription.sort()
if parallel > 1:
    # split feature
    feat = feat.sort(by="utt").subset(chunks=parallel)
    # split transcription depending on utterance IDs of each feat
    tempTrans = []
    tempAli = []
    for f in feat:
        tempTrans.append( transcription.subset(keys=f.utts) )
        tempAli.append( newAli.subset(keys=f.utts) )
    transcription = tempTrans
    newAli = tempAli

# 3. Train
print("----Train the triphone model--------------------------------------")
model.train(feat,
            transcription, 
            os.path.join(path_dataset,"dict","L.fst"), 
            tree,
            tempDir=os.path.join(path_dataset,"train_delta"),
            initialAli=newAli,
            numIters=numIters, 
            maxIterInc=maxIterInc,
            totgauss=15000,
            realignIter=realignIter,
            boostSilence=1.0,
        )
print(model.info)
# Save the tree
model.tree.save(os.path.join(path_dataset,"train_delta","tree"))
print(f"Tree has been saved.")
del feat

----Train the triphone model--------------------------------------
Start to train triphone model.
Start Time: 2021/05/23-00:12:47
Convert transcription to int value format.
Compiling training graph.
Iter >> 1
Use the provided initial alignment
Accumulate GMM statistics
Update GMM parameters
Used time: 164.2384 seconds
Iter >> 2
Skip aligning
Accumulate GMM statistics
Update GMM parameters
Used time: 161.2999 seconds
Iter >> 3
Skip aligning
Accumulate GMM statistics
Update GMM parameters
Used time: 171.5843 seconds
Iter >> 4
Skip aligning
Accumulate GMM statistics
Update GMM parameters
Used time: 168.7223 seconds
Iter >> 5
Skip aligning
Accumulate GMM statistics
Update GMM parameters
Used time: 162.1427 seconds
Iter >> 6
Skip aligning
Accumulate GMM statistics
Update GMM parameters
Used time: 169.2235 seconds
Iter >> 7
Skip aligning
Accumulate GMM statistics
Update GMM parameters
Used time: 167.6211 seconds
Iter >> 8
Skip aligning
Accumulate GMM statistics
Update GMM parameters
Used tim

In [12]:
#declare.is_file( os.path.join(path_dataset,"train_delta","final.mdl") )
#declare.is_file( os.path.join(path_dataset,"train_delta","tree") )
    
#model = exkaldi.load_hmm(os.path.join(path_dataset,"train_delta","final.mdl"))
#tree = exkaldi.load_tree(os.path.join(path_dataset,"train_delta","tree"))

# Results with CMVN and less realignIter
# ------------- Compile WFST training ---------------------- 
# Make a WFST decoding graph
make_WFST_graph(
            outDir=os.path.join(path_dataset,"train_delta","graph"),
            hmm=model,
            tree=tree,
        )
# Decode test data
GMM_decode_mfcc_and_score(
            outDir=os.path.join(path_dataset,"train_delta",f"decode_{order}grams"), 
            hmm=model,
            HCLGfile=os.path.join(path_dataset,"train_delta","graph",f"HCLG.{order}.fst"),
        )

Start to make WFST graph.
Load lexicon bank.
Generate CLG fst done.
Compose HCLG fst done.
Load test feature.
Feature type is delta. Add 2-order deltas.
Start to decode
Decode time cost:  1490.50 seconds
Generate lattice done.
Now score:
Penalty: 0.0, LMWT: 1, PER: 25.16%
Penalty: 0.0, LMWT: 2, PER: 24.95%
Penalty: 0.0, LMWT: 3, PER: 24.69%
Penalty: 0.0, LMWT: 4, PER: 24.48%
Penalty: 0.0, LMWT: 5, PER: 24.27%
Penalty: 0.0, LMWT: 6, PER: 24.04%
Penalty: 0.0, LMWT: 7, PER: 23.83%
Penalty: 0.0, LMWT: 8, PER: 23.72%
Penalty: 0.0, LMWT: 9, PER: 23.61%
Penalty: 0.0, LMWT: 10, PER: 23.62%
Penalty: 0.5, LMWT: 1, PER: 25.15%
Penalty: 0.5, LMWT: 2, PER: 24.93%
Penalty: 0.5, LMWT: 3, PER: 24.66%
Penalty: 0.5, LMWT: 4, PER: 24.44%
Penalty: 0.5, LMWT: 5, PER: 24.21%
Penalty: 0.5, LMWT: 6, PER: 23.99%
Penalty: 0.5, LMWT: 7, PER: 23.85%
Penalty: 0.5, LMWT: 8, PER: 23.68%
Penalty: 0.5, LMWT: 9, PER: 23.68%
Penalty: 0.5, LMWT: 10, PER: 23.69%
Penalty: 1.0, LMWT: 1, PER: 25.13%
Penalty: 1.0, LMWT: 2, PE

In [75]:
"""
# Resutls without CMVN (with only raw mfcc)
# ------------- Compile WFST training ---------------------- 
# Make a WFST decoding graph
make_WFST_graph(
            outDir=os.path.join(path_dataset,"train_delta","graph"),
            hmm=model,
            tree=tree,
        )
# Decode test data
GMM_decode_mfcc_and_score(
            outDir=os.path.join(path_dataset,"train_delta",f"decode_{order}grams"), 
            hmm=model,
            HCLGfile=os.path.join(path_dataset,"train_delta","graph",f"HCLG.{order}.fst"),
        )
"""

Start to make WFST graph.
Load lexicon bank.
Generate CLG fst done.
Compose HCLG fst done.
Load test feature.
Feature type is delta. Add 2-order deltas.
Start to decode
Decode time cost:  921.69 seconds
Generate lattice done.
Now score:
Penalty: 0.0, LMWT: 1, PER: 25.08%
Penalty: 0.0, LMWT: 2, PER: 24.85%
Penalty: 0.0, LMWT: 3, PER: 24.58%
Penalty: 0.0, LMWT: 4, PER: 24.3%
Penalty: 0.0, LMWT: 5, PER: 23.98%
Penalty: 0.0, LMWT: 6, PER: 23.79%
Penalty: 0.0, LMWT: 7, PER: 23.62%
Penalty: 0.0, LMWT: 8, PER: 23.5%
Penalty: 0.0, LMWT: 9, PER: 23.42%
Penalty: 0.0, LMWT: 10, PER: 23.39%
Penalty: 0.5, LMWT: 1, PER: 25.09%
Penalty: 0.5, LMWT: 2, PER: 24.82%
Penalty: 0.5, LMWT: 3, PER: 24.54%
Penalty: 0.5, LMWT: 4, PER: 24.22%
Penalty: 0.5, LMWT: 5, PER: 23.94%
Penalty: 0.5, LMWT: 6, PER: 23.76%
Penalty: 0.5, LMWT: 7, PER: 23.59%
Penalty: 0.5, LMWT: 8, PER: 23.5%
Penalty: 0.5, LMWT: 9, PER: 23.43%
Penalty: 0.5, LMWT: 10, PER: 23.41%
Penalty: 1.0, LMWT: 1, PER: 25.08%
Penalty: 1.0, LMWT: 2, PER: 2

# Train GMM-HMM model with LDA+feature

In [18]:
# ARGS
delta = 2 #(n-order to add to the feature)
numIters = 35 # Training iterations
maxIterInc = 25 # Final iteration of increasing gaussians
realignIter = [10,20,30] # iteration to allign feature
order = 5 # N-GRAM to use (min 1 | max 6)
beam = 13 # Decode beam size
latBeam = 6 # Lattice beam size
acwt = 0.083333 # Acoustic model weight
parallel = 1 # parallel process to compute feature of train dataset (min 1 | max 10)
skipTrain = False # If True, skip training. Do decoding only
splice = 3 # How many left-right frames to splice
mlltIter = [2,4,6,12] # The iteration estimate mllt matrix

In [14]:
# ------------- Prepare feature and previous alignment for training ----------------------
# 1. Load the feature for training
print(f"Load MFCC+CMVN feature.")
feat = exkaldi.load_index_table(os.path.join(path_dataset,"mfcc","train","mfcc_cmvn.ark"))
print(f"Splice {splice} frames.")
originalFeat = exkaldi.splice_feature(feat,left=splice,right=splice,outFile=os.path.join(path_dataset,"train_delta","mfcc_cmvn_splice.ark"))
# 2. Load previous alignment and lexicons
ali = exkaldi.load_index_table(os.path.join(path_dataset,"train_delta","*final.ali"),useSuffix="ark")
lexicons = exkaldi.load_lex(os.path.join(path_dataset,"dict","lexicons.lex"))
# 3. Estimate LDA matrix and convert feature
print("Estimate LDA matrix.")
exkaldi.hmm.accumulate_LDA_stats(
                ali=ali, 
                lexicons=lexicons, 
                hmm=os.path.join(path_dataset,"train_delta","final.mdl"), 
                feat=originalFeat, 
                outFile=os.path.join(path_dataset,"train_lda_mllt","ldaStats.acc"),
                silenceWeight=0,
                randPrune=4,
            )
exkaldi.hmm.estimate_LDA_matrix(
                            statsFiles=os.path.join(path_dataset,"train_lda_mllt","*ldaStats.acc"), 
                            targetDim=40, 
                            outFile=os.path.join(path_dataset,"train_lda_mllt","trans.mat")
                        )
print("Transform feature")
ldaFeat = exkaldi.transform_feat(
                        feat=originalFeat, 
                        matFile=os.path.join(path_dataset,"train_lda_mllt","trans.mat"),
                        outFile=os.path.join(path_dataset,"train_lda_mllt","lda_feat.ark"),
                    )

Load MFCC+CMVN feature.
Splice 3 frames.
Estimate LDA matrix.
Transform feature


In [15]:
# -------------- Build the decision tree ------------------------
print("Start build a tree")
tree = exkaldi.hmm.DecisionTree(lexicons=lexicons, contextWidth=3, centralPosition=1)
tree.train(
            feat=ldaFeat, 
            hmm=os.path.join(path_dataset,"train_delta","final.mdl"), 
            ali=ali,
            topoFile=os.path.join(path_dataset,"dict","topo"), 
            numLeaves=2500,
            tempDir=os.path.join(path_dataset,"train_lda_mllt"), 
        )
tree.save(os.path.join(path_dataset,"train_delta","tree"))
print(f"Build tree done.")
del ldaFeat

# ------------- Start training ----------------------
# 1. Initialize a triphone HMM object
print("Initialize a triphone HMM object")
model = exkaldi.hmm.TriphoneHMM(lexicons=lexicons)
model.initialize(
            tree=tree, 
            topoFile=os.path.join(path_dataset,"dict","topo"),
            treeStatsFile=os.path.join(path_dataset,"train_lda_mllt","treeStats.acc"), 
        )    
print(f"Initialized a monophone HMM-GMM model: {model.info}.")

# 2. convert the previous alignment
print(f"Transform the alignment")
newAli = exkaldi.hmm.convert_alignment(
                                ali=ali,
                                originHmm=os.path.join(path_dataset,"train_delta","final.mdl"), 
                                targetHmm=model, 
                                tree=tree,
                                outFile=os.path.join(path_dataset,"train_lda_mllt","initial.ali"),
                            )

Start build a tree
Start to build decision tree.
Start Time: 2021/05/22-15:21:02
Accumulate tree statistics
Cluster phones and compile questions
Build tree
Done to build the decision tree.
Saved Final Tree: /home/marrakchi/TFM/TFM_env/Datasets/ExKaldiASRDatasetFormat/train_lda_mllt/tree
End Time: 2021/05/22-15:24:22
Build tree done.
Initialize a triphone HMM object
Initialized a monophone HMM-GMM model: GmmHmmInfo(phones=26, pdfs=2160, transitionIds=4422, transitionStates=2203, dimension=40, gaussians=2160).
Transform the alignment


In [16]:
# 2. Split data for parallel training
transcription = exkaldi.load_transcription(os.path.join(path_dataset,"data","train","text"))
transcription = transcription.sort()
if parallel > 1:
    # split feature
    originalFeat = originalFeat.sort(by="utt").subset(chunks=parallel)
    # split transcription depending on utterance IDs of each feat
    tempTrans = []
    tempAli = []
    for f in originalFeat:
        tempTrans.append( transcription.subset(keys=f.utts) )
        tempAli.append( newAli.subset(keys=f.utts) )
    transcription = tempTrans
    newAli = tempAli

# 3. Train
print("Train the triphone model")
model.train(
            originalFeat,
            transcription, 
            os.path.join(path_dataset,"dict","L.fst"), 
            tree,
            tempDir=os.path.join(path_dataset,"train_lda_mllt"),
            initialAli=newAli,
            ldaMatFile=os.path.join(path_dataset,"train_lda_mllt","trans.mat"),
            numIters=numIters, 
            maxIterInc=maxIterInc,
            totgauss=15000,
            realignIter=realignIter,
            mlltIter=mlltIter,
        )
print(model.info)
# Save the tree
del originalFeat
del newAli

Train the triphone model
Do LDA + MLLT training.
Start to train triphone model.
Start Time: 2021/05/22-15:34:48
Convert transcription to int value format.
Compiling training graph.
Iter >> 1
Use the provided initial alignment
Skip tansform feature
Accumulate GMM statistics
Update GMM parameters
Used time: 198.4770 seconds
Iter >> 2
Skip aligning
Accumulate MLLT statistics
Estimate MLLT matrix
Transform GMM means
Compose new LDA-MLLT transform matrix
Transform feature
Accumulate GMM statistics
Update GMM parameters
Used time: 877.7089 seconds
Iter >> 3
Skip aligning
Skip tansform feature
Accumulate GMM statistics
Update GMM parameters
Used time: 181.1236 seconds
Iter >> 4
Skip aligning
Accumulate MLLT statistics
Estimate MLLT matrix
Transform GMM means
Compose new LDA-MLLT transform matrix
Transform feature
Accumulate GMM statistics
Update GMM parameters
Used time: 908.5012 seconds
Iter >> 5
Skip aligning
Skip tansform feature
Accumulate GMM statistics
Update GMM parameters
Used time: 1

In [19]:
declare.is_file( os.path.join(path_dataset,"train_lda_mllt","final.mdl") )
declare.is_file( os.path.join(path_dataset,"train_lda_mllt","tree") )
    
model = exkaldi.load_hmm(os.path.join(path_dataset,"train_lda_mllt","final.mdl"))
tree = exkaldi.load_tree(os.path.join(path_dataset,"train_lda_mllt","tree"))
        
# ------------- Compile WFST training ----------------------
# Make a WFST decoding graph
make_WFST_graph(
            outDir=os.path.join(path_dataset,"train_lda_mllt","graph"),
            hmm=model,
            tree=tree,
        )
# Decode test data
GMM_decode_mfcc_and_score(
            outDir=os.path.join(path_dataset,"train_lda_mllt",f"decode_{order}grams"), 
            hmm=model,
            HCLGfile=os.path.join(path_dataset,"train_lda_mllt","graph",f"HCLG.{order}.fst"),
            tansformMatFile=os.path.join(path_dataset,"train_lda_mllt","trans.mat"),
        )


Start to make WFST graph.
Load lexicon bank.
Generate CLG fst done.
Compose HCLG fst done.
Load test feature.
Feature type is lda+mllt. Transform feature.
Start to decode
Decode time cost:  1271.29 seconds
Generate lattice done.
Now score:
Penalty: 0.0, LMWT: 1, PER: 24.42%
Penalty: 0.0, LMWT: 2, PER: 24.16%
Penalty: 0.0, LMWT: 3, PER: 23.92%
Penalty: 0.0, LMWT: 4, PER: 23.67%
Penalty: 0.0, LMWT: 5, PER: 23.45%
Penalty: 0.0, LMWT: 6, PER: 23.31%
Penalty: 0.0, LMWT: 7, PER: 23.12%
Penalty: 0.0, LMWT: 8, PER: 23.02%
Penalty: 0.0, LMWT: 9, PER: 22.93%
Penalty: 0.0, LMWT: 10, PER: 22.87%
Penalty: 0.5, LMWT: 1, PER: 24.41%
Penalty: 0.5, LMWT: 2, PER: 24.13%
Penalty: 0.5, LMWT: 3, PER: 23.88%
Penalty: 0.5, LMWT: 4, PER: 23.62%
Penalty: 0.5, LMWT: 5, PER: 23.41%
Penalty: 0.5, LMWT: 6, PER: 23.24%
Penalty: 0.5, LMWT: 7, PER: 23.13%
Penalty: 0.5, LMWT: 8, PER: 23.02%
Penalty: 0.5, LMWT: 9, PER: 22.94%
Penalty: 0.5, LMWT: 10, PER: 22.86%
Penalty: 1.0, LMWT: 1, PER: 24.39%
Penalty: 1.0, LMWT: 2, 

# Train a GMM-HMM model with fmllr feature.
## --> Speaker Adapted Training (SAT)

In [7]:
# ARGS
delta = 2 #(n-order to add to the feature)
numIters = 35 # Training iterations
maxIterInc = 25 # Final iteration of increasing gaussians
realignIter = [10,20,30] # iteration to allign feature
order = 5 # N-GRAM to use (min 1 | max 6)
beam = 13 # Decode be                                          am size
latBeam = 6 # Lattice beam size
acwt = 0.083333 # Acoustic model weight
parallel = 1 # parallel process to compute feature of train dataset (min 1 | max 10)
skipTrain = False # If True, skip training. Do decoding only
splice = 3 # How many left-right frames to splice
fmllrIter = [2,4,6,12] # The iteration to estimate fmllr matrix

In [8]:
# ------------- Prepare feature and previous alignment for training ----------------------
# 1. Load the feature for training
print(f"Load MFCC+CMVN feature.")
feat = exkaldi.load_index_table(os.path.join(path_dataset,"mfcc","train","mfcc_cmvn.ark"))
print(f"Splice {splice} frames.")
originalFeat = exkaldi.splice_feature(feat,left=splice,right=splice,outFile=os.path.join(path_dataset,"train_delta","mfcc_cmvn_splice.ark"))
print(f"Transform LDA feature")
ldaFeat = exkaldi.transform_feat(
                        feat=originalFeat, 
                        matFile=os.path.join(path_dataset,"train_lda_mllt","trans.mat"),
                        outFile=os.path.join(path_dataset,"train_sat","lda_feat.ark"),
                    )
del originalFeat
# 2. Load previous alignment and lexicons
ali = exkaldi.load_index_table(os.path.join(path_dataset,"train_lda_mllt","*final.ali"),useSuffix="ark")
lexicons = exkaldi.load_lex(os.path.join(path_dataset,"dict","lexicons.lex"))
# 3. Estimate the primary fMLLR transform matrix
print("Estiminate the primary fMLLR transform matrixs")


Load MFCC+CMVN feature.
Splice 3 frames.
Transform LDA feature
Estiminate the primary fMLLR transform matrixs


In [9]:
fmllrTransMat = exkaldi.hmm.estimate_fMLLR_matrix(
                                            aliOrLat=ali,
                                            lexicons=lexicons, 
                                            aliHmm=os.path.join(path_dataset,"train_lda_mllt","final.mdl"), 
                                            feat=ldaFeat,
                                            spk2utt=os.path.join(path_dataset,"data","train","spk2utt"),
                                            outFile=os.path.join(path_dataset,"train_sat","trans.ark"),
                                        )
print("Transform feature")
fmllrFeat = exkaldi.use_fmllr(
                        ldaFeat,
                        fmllrTransMat,
                        utt2spk=os.path.join(path_dataset,"data","train","utt2spk"),
                        outFile=os.path.join(path_dataset,"train_sat","fmllr_feat.ark"),
                    )

Transform feature


In [10]:
# -------------- Build the decision tree ------------------------
print("Start build a tree")
tree = exkaldi.hmm.DecisionTree(lexicons=lexicons, contextWidth=3, centralPosition=1)
tree.train(
            feat=fmllrFeat, 
            hmm=os.path.join(path_dataset,"train_lda_mllt","final.mdl"), 
            ali=ali,
            topoFile=os.path.join(path_dataset,"dict","topo"), 
            numLeaves=2500,
            tempDir=os.path.join(path_dataset,"train_sat"), 
        )
tree.save(os.path.join(path_dataset,"train_sat","tree"))
print(f"Build tree done.")
del fmllrFeat

# ------------- Start training ----------------------
# 1. Initialize a triphone HMM object
print("Initialize a triphone HMM object")
model = exkaldi.hmm.TriphoneHMM(lexicons=lexicons)
model.initialize(
            tree=tree, 
            topoFile=os.path.join(path_dataset,"dict","topo"),
            treeStatsFile=os.path.join(path_dataset,"train_sat","treeStats.acc"), 
        )    
print(f"Initialized a monophone HMM-GMM model: {model.info}.")

# 2. convert the previous alignment
print(f"Transform the alignment")
newAli = exkaldi.hmm.convert_alignment(
                                ali=ali,
                                originHmm=os.path.join(path_dataset,"train_lda_mllt","final.mdl"), 
                                targetHmm=model, 
                                tree=tree,
                                outFile=os.path.join(path_dataset,"train_sat","initial.ali"),
                            )

# 2. Split data for parallel training
transcription = exkaldi.load_transcription(os.path.join(path_dataset,"data","train","text"))
transcription = transcription.sort()

if parallel > 1:
    # split feature
    ldaFeat = ldaFeat.sort(by="utt").subset(chunks=parallel)
    # split transcription depending on utterance IDs of each feat
    tempTrans = []
    tempAli = []
    tempFmllrMat = []
    for f in ldaFeat:
        tempTrans.append( transcription.subset(keys=f.utts) )
        tempAli.append( newAli.subset(keys=f.utts) )
        spks = exkaldi.utt_to_spk(f.utts, utt2spk=os.path.join(path_dataset,"data","train","utt2spk"))
        tempFmllrMat.append( fmllrTransMat.subset(keys=spks) )
    transcription = tempTrans
    newAli = tempAli
    fmllrTransMat = tempFmllrMat

Start build a tree
Start to build decision tree.
Start Time: 2021/05/23-16:12:15
Accumulate tree statistics
Cluster phones and compile questions
Build tree
Done to build the decision tree.
Saved Final Tree: /home/marrakchi/TFM/TFM_env/Datasets/ExKaldiASRDatasetFormat/train_sat/tree
End Time: 2021/05/23-16:15:32
Build tree done.


In [13]:
# 3. Train
print("Train the triphone model")
model.train(
            ldaFeat,
            transcription, 
            os.path.join(path_dataset,"dict","L.fst"), 
            tree,
            tempDir=os.path.join(path_dataset,"train_sat"),
            initialAli=newAli,
            fmllrTransMat=fmllrTransMat,
            spk2utt=os.path.join(path_dataset,"data","train","spk2utt"), 
            utt2spk=os.path.join(path_dataset,"data","train","utt2spk"),                
            numIters=numIters, 
            maxIterInc=maxIterInc,
            totgauss=15000,
            realignIter=realignIter,
            fmllrIter=fmllrIter,
            boostSilence=1.0,
            power=0.2,
            fmllrSilWt=0.0,       
        )
print(model.info)
del ldaFeat
del fmllrTransMat
del newAli

Train the triphone model
Do SAT. Transform to fMLLR feature. <spk2utt> and <utt2spk> files are necessary in this case.
Start to train triphone model.
Start Time: 2021/05/23-16:51:00
Convert transcription to int value format.
Compiling training graph.
Iter >> 1
Use the provided initial alignment
Skip tansforming feature
Accumulate GMM statistics
Update GMM parameters
Used time: 199.8287 seconds
Iter >> 2
Skip aligning
Estimate fMLLR matrix
Transform feature
Accumulate GMM statistics
Update GMM parameters
Used time: 1605.5952 seconds
Iter >> 3
Skip aligning
Skip tansforming feature
Accumulate GMM statistics
Update GMM parameters
Used time: 199.4587 seconds
Iter >> 4
Skip aligning
Estimate fMLLR matrix
Transform feature
Accumulate GMM statistics
Update GMM parameters
Used time: 1618.5550 seconds
Iter >> 5
Skip aligning
Skip tansforming feature
Accumulate GMM statistics
Update GMM parameters
Used time: 198.1842 seconds
Iter >> 6
Skip aligning
Estimate fMLLR matrix
Transform feature
Accumul

In [8]:
declare.is_file( os.path.join(path_dataset,"train_sat","final.mdl") )
declare.is_file( os.path.join(path_dataset,"train_sat","tree") )
model = exkaldi.load_hmm( os.path.join(path_dataset,"train_sat","final.mdl") )
tree = exkaldi.load_tree( os.path.join(path_dataset,"train_sat","tree") )

In [9]:
# ------------- Compile WFST training ----------------------
# Make a WFST decoding graph
make_WFST_graph(
            outDir=os.path.join(path_dataset,"train_sat","graph"),
            hmm=model,
            tree=tree,
        )
# Decode test data
GMM_decode_fmllr_and_score(
                    outDir=os.path.join(path_dataset,"train_sat",f"decode_{order}grams"), 
                    hmm=model,
                    HCLGfile=os.path.join(path_dataset,"train_sat","graph",f"HCLG.{order}.fst"),
                    tansformMatFile=os.path.join(path_dataset,"train_lda_mllt","trans.mat"),
                )

Start to make WFST graph.
Load lexicon bank.
Generate CLG fst done.
Compose HCLG fst done.
Load test feature.
Feature type is lda+mllt
Transform feature
Decode the first time with original feature.
Estimate the primary fMLLR transform matrix.
Transform feature with primary matrix.
Decode the second time with primary fmllr feature.
Determinize secondary lattice.
Estimate the secondary fMLLR transform matrix.
Compose the primary and secondary transform matrix.
Transform feature with final matrix.
Rescore secondary lattice.
Determinize secondary lattice.
Generate lattice done.
Generate reference text done.
Now score:
Penalty: 0.0, LMWT: 1, PER: 24.4%
Penalty: 0.0, LMWT: 2, PER: 24.14%
Penalty: 0.0, LMWT: 3, PER: 23.89%
Penalty: 0.0, LMWT: 4, PER: 23.65%
Penalty: 0.0, LMWT: 5, PER: 23.46%
Penalty: 0.0, LMWT: 6, PER: 23.25%
Penalty: 0.0, LMWT: 7, PER: 23.08%
Penalty: 0.0, LMWT: 8, PER: 22.92%
Penalty: 0.0, LMWT: 9, PER: 22.81%
Penalty: 0.0, LMWT: 10, PER: 22.74%
Penalty: 0.5, LMWT: 1, PER: 

# Train a DNN acoustic model with fmllr feature.

In [6]:
# ARGS
LDAsplice = 3 # Splice how many frames to head and tail for LDA feature. 
randomSeed = 1234 # Random seed
batchSize = 128 # Mini btach size
gu = "all" # Use GPU
epoch = 30 # Epoches
testStartEpoch = 5 #Start to evaluate test dataset.
dropout = 0.2
useCMVN = False #Wether apply CMVN to fmllr feature.
splice = 10 # Splice how many frames to head and tail for Fmllr feature.
delta = 2 #Wether add delta to fmllr feature.
normalizeFeat = True # Wether to normalize the chunk dataset
normalizeAMP = False
order = 5 # N-GRAM to use (min 1 | max 6)
beam = 13 # Decode beam size
latBeam = 6 # Lattice beam size
acwt = 0.083333 # Acoustic model weight
predictModel = "" # If not void, skip training. Do decoding only.

In [7]:
def prepare_DNN_data():
    print("Start to prepare data for DNN training")
    assert os.path.isdir(f"{path_dataset}/train_sat"), "Please run previous programs up to SAT training."

    # Lexicons and Gmm-Hmm model
    lexicons = exkaldi.load_lex( f"{path_dataset}/dict/lexicons.lex" )
    hmm = f"{path_dataset}/train_sat/final.mdl"
    tree = f"{path_dataset}/train_sat/tree"

    for Name in ["train","dev","test"]:

        exkaldi.utils.make_dependent_dirs(f"{path_dataset}/train_dnn/data/{Name}", pathIsFile=False)
        # Make LDA feature
        print(f"Make LDA feature for '{Name}'")
        feat = exkaldi.load_feat( f"{path_dataset}/mfcc/{Name}/mfcc_cmvn.ark" )
        feat = feat.splice(left=LDAsplice, right=LDAsplice)
        feat = exkaldi.transform_feat(feat, matFile=f"{path_dataset}/train_lda_mllt/trans.mat" )
        # Compile the aligning graph
        print(f"Compile aligning graph")
        transInt = exkaldi.hmm.transcription_to_int(
                                                  transcription=f"{path_dataset}/data/{Name}/text",
                                                  symbolTable=lexicons("words"),
                                                  unkSymbol=lexicons("oov"),
                                                )
        graphFile = exkaldi.decode.wfst.compile_align_graph(
                                            hmm,
                                            tree, 
                                            transcription=transInt,
                                            LFile= f"{path_dataset}/dict/L.fst",
                                            outFile=f"{path_dataset}/train_dnn/data/{Name}/align_graph",
                                            lexicons=lexicons,
                                        )
        # Align first time
        print(f"Align the first time")
        ali = exkaldi.decode.wfst.gmm_align(
                                        hmm,
                                        feat, 
                                        alignGraphFile=graphFile, 
                                        lexicons=lexicons,
                                    )
        # Estimate transform matrix
        print(f"Estimate fMLLR transform matrix")
        fmllrTransMat = exkaldi.hmm.estimate_fMLLR_matrix(
                                    aliOrLat=ali,
                                    lexicons=lexicons, 
                                    aliHmm=hmm, 
                                    feat=feat,
                                    spk2utt=f"{path_dataset}/data/{Name}/spk2utt",
                                )
        fmllrTransMat.save( f"{path_dataset}/train_dnn/data/{Name}/trans.ark" )
        # Transform feature
        print(f"Transform feature")
        feat = exkaldi.use_fmllr(
                            feat,
                            fmllrTransMat,
                            utt2spk=f"{path_dataset}/data/{Name}/utt2spk",
                        )
        # Align second time with new feature
        print(f"Align the second time")
        ali = exkaldi.decode.wfst.gmm_align(
                                        hmm,
                                        feat,
                                        alignGraphFile=graphFile, 
                                        lexicons=lexicons,
                                    )
        # Save alignment and feature
        print(f"Save final fmllr feature and alignment")
        feat.save( f"{path_dataset}/train_dnn/data/{Name}/fmllr.ark" )
        ali.save( f"{path_dataset}/train_dnn/data/{Name}/ali" ) 
        # Transform alignment
        print(f"Generate pdf ID and phone ID alignment")
        ali.to_numpy(aliType="pdfID",hmm=hmm).save( f"{path_dataset}/train_dnn/data/{Name}/pdfID.npy" )
        ali.to_numpy(aliType="phoneID",hmm=hmm).save( f"{path_dataset}/train_dnn/data/{Name}/phoneID.npy" )
        del ali
        # Compute cmvn for fmllr feature
        print(f"Compute the CMVN for fmllr feature")
        cmvn = exkaldi.compute_cmvn_stats(feat, spk2utt=f"{path_dataset}/data/{Name}/spk2utt")
        cmvn.save( f"{path_dataset}/train_dnn/data/{Name}/cmvn_of_fmllr.ark" )
        del cmvn
        del feat
        # copy spk2utt utt2spk and text file
        shutil.copyfile( f"{path_dataset}/data/{Name}/spk2utt", f"{path_dataset}/train_dnn/data/{Name}/spk2utt")
        shutil.copyfile( f"{path_dataset}/data/{Name}/utt2spk", f"{path_dataset}/train_dnn/data/{Name}/utt2spk")
        shutil.copyfile( f"{path_dataset}/data/{Name}/text", f"{path_dataset}/train_dnn/data/{Name}/text" )
        transInt.save( f"{path_dataset}/data/{Name}/text.int" )

    print("Write feature and alignment dim information")
    dims = exkaldi.ListTable()
    feat = exkaldi.load_feat( f"{path_dataset}/train_dnn/data/test/fmllr.ark" ) 
    dims["fmllr"] = feat.dim
    del feat
    hmm = exkaldi.hmm.load_hmm( f"{path_dataset}/train_sat/final.mdl" )
    dims["phones"] = hmm.info.phones + 1
    dims["pdfs"] = hmm.info.pdfs
    del hmm
    dims.save( f"{path_dataset}/train_dnn/data/dims" )

In [8]:
def process_feat_ali(training=True):

    if training:
        Name = "train"
    else:
        Name = "dev"

    feat = exkaldi.load_feat( f"{path_dataset}/train_dnn/data/{Name}/fmllr.ark" )

    if useCMVN:
        cmvn = exkaldi.load_cmvn(f"{path_dataset}/train_dnn/data/{Name}/cmvn_of_fmllr.ark")
        feat = exkaldi.use_cmvn(feat,cmvn,f"{path_dataset}/train_dnn/data/{Name}/utt2spk")
        del cmvn
    
    if delta > 0:
        feat = feat.add_delta(delta)

    if splice > 0:
        feat = feat.splice(splice)

    feat = feat.to_numpy()

    if normalizeFeat:
        feat = feat.normalize(std=True)
    
    pdfAli = exkaldi.load_ali( f"{path_dataset}/train_dnn/data/{Name}/pdfID.npy" )
    phoneAli = exkaldi.load_ali( f"{path_dataset}/train_dnn/data/{Name}/phoneID.npy" )
    
    feat.rename("feat")
    pdfAli.rename("pdfID")
    phoneAli.rename("phoneID")

    dataset = exkaldi.tuple_dataset([feat, pdfAli, phoneAli], frameLevel=True)
    random.shuffle(dataset)

    return dataset

In [9]:
def make_generator(dataset):

    dataIndex = 0
    datasetSize = len(dataset)
    while True:
        if dataIndex >= datasetSize:
            random.shuffle(dataset)
            dataIndex = 0
        one = dataset[dataIndex]
        dataIndex += 1
        yield one.feat[0], {"pdfID":one.pdfID[0], "phoneID":one.phoneID[0]}

def prepare_test_data(postProbDim):

    feat = exkaldi.load_feat( f"{path_dataset}/train_dnn/data/test/fmllr.ark" )

    if useCMVN:
        cmvn = exkaldi.load_cmvn( f"{path_dataset}/train_dnn/data/test/cmvn_of_fmllr.ark" )
        feat = exkaldi.use_cmvn(feat, cmvn, utt2spk=f"{path_dataset}/train_dnn/data/test/utt2spk")
        del cmvn

    if delta > 0:
        feat = feat.add_delta(delta)

    if splice > 0:
        feat = feat.splice(splice)

    feat = feat.to_numpy()
    if normalizeFeat:
        feat = feat.normalize(std=True)

    # Normalize acoustic model output
    if normalizeAMP:
        ali = exkaldi.load_ali(f"{path_dataset}/train_dnn/data/train/pdfID.npy", aliType="pdfID")
        normalizeBias = exkaldi.nn.compute_postprob_norm(ali,postProbDim)
    else:
        normalizeBias = 0
    
    # ref transcription
    trans = exkaldi.load_transcription(f"{path_dataset}/train_dnn/data/test/text")
    convertTable = exkaldi.load_list_table(path_dataset+'word2phone.map')
    trans = trans.convert(convertTable, unkSymbol='<UNK>')   
    
    return feat, normalizeBias, trans

In [10]:
def make_DNN_model(inputDim,outDimPdf,outDimPho):
    
    inputs = keras.Input((inputDim,))

    ln1 = keras.layers.Dense(1024, activation=None, use_bias=False)(inputs)
    ln1_bn = keras.layers.BatchNormalization(momentum=0.95)(ln1)
    ln1_ac = keras.layers.ReLU()(ln1_bn)
    ln1_do = keras.layers.Dropout(dropout)(ln1_ac)

    ln2 = keras.layers.Dense(1024, activation=None, use_bias=False)(ln1_do)
    ln2_bn = keras.layers.BatchNormalization(momentum=0.95)(ln2)
    ln2_ac = keras.layers.ReLU()(ln2_bn)
    ln2_do = keras.layers.Dropout(dropout)(ln2_ac)

    ln3 = keras.layers.Dense(1024, activation=None, use_bias=False)(ln2_do)
    ln3_bn = keras.layers.BatchNormalization(momentum=0.95)(ln3)
    ln3_ac = keras.layers.ReLU()(ln3_bn)
    ln3_do = keras.layers.Dropout(dropout)(ln3_ac)

    ln4 = keras.layers.Dense(1024, activation=None, use_bias=False)(ln3_do)
    ln4_bn = keras.layers.BatchNormalization(momentum=0.95)(ln4)
    ln4_ac = keras.layers.ReLU()(ln4_bn)
    ln4_do = keras.layers.Dropout(dropout)(ln4_ac)

    ln5 = keras.layers.Dense(1024, activation=None, use_bias=False)(ln4_do)
    ln5_bn = keras.layers.BatchNormalization(momentum=0.95)(ln5)
    ln5_ac = keras.layers.ReLU()(ln5_bn)
    ln5_do = keras.layers.Dropout(dropout)(ln5_ac)

    outputs_pdf = keras.layers.Dense(outDimPdf,activation=None,use_bias=True,kernel_initializer="he_normal", bias_initializer='zeros', name="pdfID")(ln5_do)

    outputs_pho = keras.layers.Dense(outDimPho,activation=None,use_bias=True,kernel_initializer="he_normal", bias_initializer='zeros', name="phoneID")(ln5_do)

    return keras.Model(inputs=inputs, outputs=[outputs_pdf,outputs_pho])


In [11]:
def decode_score_test(prob,trans,outDir):

    trans.save( os.path.join(outDir,"ref.txt") )

    hmmFile = f"{path_dataset}/train_sat/final.mdl"
    HCLGFile = f"{path_dataset}/train_sat/graph/HCLG.{path_dataset}.fst"
    lexicons = exkaldi.load_lex( f"{path_dataset}/dict/lexicons.lex" )
    #phoneMap = exkaldi.load_list_table( f"{args.expDir}/dict/phones.48_to_39.map" )
    phoneMap = exkaldi.load_list_table(path_dataset+'word2phone.map')
    #print("Decoding...")
    lat = exkaldi.decode.wfst.nn_decode(
                                  prob=prob, 
                                  hmm=hmmFile, 
                                  HCLGFile=HCLGFile, 
                                  symbolTable=lexicons("words"),
                                  beam=beam,
                                  latBeam=latBeam,
                                  acwt=acwt,
                                  minActive=200,
                                  maxActive=7000,
                              )
    #print("Score...")
    minWER = None
    for penalty in [0.,0.5,1.0]:
        for LMWT in range(1,15,1):

            newLat = lat.add_penalty(penalty)
            result = newLat.get_1best(lexicons("phones"),hmmFile,lmwt=LMWT,acwt=1,phoneLevel=True)
            result = exkaldi.hmm.transcription_from_int(result,lexicons("phones"))
            result = result.convert(phoneMap, unkSymbol='<UNK>' )
            fileName = f"{outDir}/penalty_{penalty}_lmwt_{LMWT}.txt"
            result.save( fileName )
            score = exkaldi.decode.score.wer(ref=trans, hyp=result, mode="present")
            if minWER == None or score.WER < minWER[0]:
                minWER = (score.WER, fileName)
            #print(f"{penalty} {LMWT}",score)

    with open( f"{outDir}/best_PER", "w") as fw:
        fw.write( f"{minWER[0]}% {minWER[1]}" )

    return minWER[0]

In [12]:
class EvaluateWER(keras.callbacks.Callback):

    def __init__(self, model, feat, bias, trans, outDir):
        exkaldi.utils.make_dependent_dirs(outDir, False)
        self.model = model
        self.feat = feat
        self.bias = bias
        self.trans = trans
        self.outDir = outDir

    def on_epoch_end(self, epoch, logs={}):

        if epoch >= testStartEpoch:
            subOutDir = os.path.join( self.outDir, f"decode_ep{epoch+1}" )
            exkaldi.utils.make_dependent_dirs(subOutDir, False)

            prob = {}
            for utt, matrix in self.feat.items():
                predPdf, predPhone = self.model(matrix, training=False)
                prob[utt] = predPdf.numpy() + self.bias
                
            prob = exkaldi.load_prob(prob)
            prob = prob.map(lambda x:exkaldi.nn.log_softmax(x,axis=1))
            prob = prob.to_bytes()

            WER = decode_score_test(prob, self.trans, subOutDir)

            logs['test_WER'] = WER

class ModelSaver(keras.callbacks.Callback):

    def __init__(self, model, outDir):

        self.model = model
        self.outDir = outDir
    
    def on_epoch_end(self, epoch, logs={}):
        
        outDir = os.path.join(self.outDir,f"model_ep{epoch+1}.h5")
        self.model.save_weights(outDir)

In [13]:
def output_probability():

    declare.is_file(predictModel)

    dims = exkaldi.load_list_table( f"{path_dataset}/train_dnn/data/dims" )
    featDim = int(dims["fmllr"])
    pdfDim = int(dims["pdfs"])
    phoneDim = int(dims["phones"])

    # Initialize model
    if delta > 0:
        featDim *= (delta+1)
    if splice > 0:
        featDim *= (2*splice+1)

    model = make_DNN_model(featDim,pdfDim,phoneDim) 
    model.load_weights(predictModel)
    print(f"Restorage model from: {predictModel}")

    for Name in ["train","test","dev"]:
        print(f"Processing: {Name} dataset")
        feat = exkaldi.load_feat( f"{path_dataset}/train_dnn/data/{Name}/fmllr.ark" )

        if useCMVN:
            print("Apply CMVN")
            cmvn = exkaldi.load_cmvn( f"{path_dataset}/train_dnn/data/{Name}/cmvn_of_fmllr.ark" )
            feat = exkaldi.use_cmvn(feat, cmvn, utt2spk=f"{path_dataset}/train_dnn/data/{Name}/utt2spk")
            del cmvn

        if delta > 0:
            print("Add delta to feature")
            feat = feat.add_delta(delta)

        if splice > 0:
            print("Splice feature")
            feat = feat.splice(splice)

        feat = feat.to_numpy()
        if normalizeFeat:
            print("Normalize")
            feat = feat.normalize(std=True)

        outProb = {}
        print("Forward model...")
        for utt,mat in feat.items():
            predPdf, predPhone = model(mat, training=False)
            outProb[utt] = exkaldi.nn.log_softmax(predPdf.numpy(),axis=1)

        #outProb = exkaldi.load_prob(outProb)
        #outProb.save(f"{args.expDir}/train_dnn/prob/{Name}.npy")
        outProb = exkaldi.load_prob(outProb).to_bytes()
        outProb.save(f"{path_dataset}/train_dnn/prob/{Name}.ark") 
        print("Save done!")

In [ ]:
random.seed(randomSeed)
np.random.seed(randomSeed)
tf.random.set_seed(randomSeed)
# ------------- Prepare data for dnn training ----------------------
print('1')
if not os.path.isfile(f"./{path_dataset}/train_dnn/data/dims"):
    prepare_DNN_data()
print('2')
# ------------- Prepare data for dnn training ----------------------
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
outDir = f"{path_dataset}/train_dnn/out_{stamp}"
exkaldi.utils.make_dependent_dirs(outDir, pathIsFile=False)
print('3')

#------------------------ Training and Validation -----------------------------
dims = exkaldi.load_list_table( f"{path_dataset}/train_dnn/data/dims" )
print('4')
featDim = int(dims["fmllr"])
print('5')
pdfDim = int(dims["pdfs"])
phoneDim = int(dims["phones"])

print('6')
  # Initialize model
if delta > 0:
    featDim *= (delta+1)
if splice > 0:
    featDim *= (2*splice+1)

print('7')
if len(predictModel.strip()) == 0:
    print('Prepare Data Iterator...')
    # Prepare fMLLR feature files
    trainDataset = process_feat_ali(training=True)
    traindataLen = len(trainDataset)
    train_gen = tf.data.Dataset.from_generator(
                                        lambda: make_generator(trainDataset),
                                        (tf.float32, {"pdfID":tf.int32,"phoneID":tf.int32})
                                ).batch(batchSize).prefetch(3)
    steps_per_epoch = traindataLen//batchSize

    devDataset = process_feat_ali(training=False)
    devdataLen = len(devDataset)
    dev_gen = tf.data.Dataset.from_generator(
                                        lambda: make_generator(devDataset),
                                        (tf.float32, {"pdfID":tf.int32,"phoneID":tf.int32})
                                ).batch(batchSize).prefetch(3)
    validation_steps = devdataLen//batchSize

    print('Prepare test data')
    testFeat, testBias, testTrans = prepare_test_data(postProbDim=pdfDim)
    
    def train_step():
        
        model = make_DNN_model(featDim,pdfDim,phoneDim)
        model.summary()

        model.compile(
                    loss = {
                            "pdfID":keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                            "phoneID":keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                        },
                    loss_weights = {"pdfID":1,"phoneID":1},
                    metrics = {
                                "pdfID":keras.metrics.SparseCategoricalAccuracy(),
                                "phoneID":keras.metrics.SparseCategoricalAccuracy(),
                            },
                    optimizer = keras.optimizers.SGD(0.08,momentum=0.0),
                )

        def lrScheduler(epoch):
            if epoch > 25:
                return 0.001
            elif epoch > 22:
                return 0.0025
            elif epoch > 19:
                return 0.005
            elif epoch > 17:
                return 0.01
            elif epoch > 15:
                return 0.02
            elif epoch > 10:
                return 0.04
            else:
                return 0.08

        model.fit(
                x = train_gen,
                steps_per_epoch=steps_per_epoch,
                epochs=epoch,

                validation_data=dev_gen,
                validation_steps=validation_steps,
                verbose=1,

                initial_epoch=0,
                callbacks=[
                            keras.callbacks.EarlyStopping(patience=5, verbose=1),
                            keras.callbacks.TensorBoard(log_dir=outDir),
                            keras.callbacks.LearningRateScheduler(lrScheduler),
                            EvaluateWER(model,testFeat,testBias,testTrans,outDir), 
                            ModelSaver(model,outDir),         
                        ],
                    )

    print("Using GPU: ", gpu)
    if gpu != "all":
        os.environ["CUDA_VISIBLE_DEVICES"] = gpu
        train_step()

    else:
        my_strategy = tf.distribute.MirroredStrategy()
        with my_strategy.scope():
            train_step()

else:
    declare.is_file(predictModel)

    model = make_DNN_model(featDim,pdfDim,phoneDim)
    model.summary()

    model.load_weights(predictModel)

    print('Prepare test data')
    testFeat, testBias, testTrans = prepare_test_data(postProbDim=pdfDim)
    scorer = EvaluateWER(model,testFeat,testBias,testTrans,outDir)

    logs = {}
    scorer.on_epoch_end(5,logs)

1
Start to prepare data for DNN training
Make LDA feature for 'train'
